In [1]:
import pandas as pd


In [188]:
val = pd.read_csv("../data/toronto_valentine_20190210_20190216.csv")
hoc = pd.read_csv("../data/korea_toronto_hocky_20190410_20190424.csv")


In [17]:
hoc[hoc.bar_id == 19088].order_id.value_counts() 

521095549    375
513690152     60
513558466     60
521765889     55
519022853     47
513558528     41
519021838     36
521911713     36
521943786     34
513690401     32
522778513     31
525659494     30
524542006     30
523722790     30
524459282     30
525659502     29
522779177     29
519022596     28
524221650     28
523539432     27
521609487     24
521068167     24
522779124     24
512704015     23
513462991     23
513690475     23
520253648     22
512806163     22
514690232     22
525623573     22
            ... 
524442311      1
512806168      1
525522783      1
512570122      1
522554472      1
513335779      1
512742411      1
523336710      1
519023098      1
513325747      1
519019834      1
519632431      1
519023097      1
512855584      1
521588724      1
512704022      1
514472272      1
524853696      1
513335776      1
522599902      1
522633575      1
521096512      1
514472296      1
514492365      1
521095550      1
511233432      1
511233424      1
522697104     

In [ ]:
Lets remove those that have a guest count of zeros

In [191]:
data = hoc[~(hoc.guest_count == 0)]
data_val = val[~(val.guest_count == 0)]

In [194]:
len(hoc)

645787

In [190]:
len(data)

478447

In [192]:
len(val)

221888

In [193]:
len(data_val)

169351

In [195]:
data.guest_count.value_counts(normalize=True) #cool

1      0.369362
2      0.281296
3      0.126547
4      0.086572
5      0.042671
6      0.027123
7      0.014917
8      0.010829
9      0.007809
10     0.005856
11     0.004642
12     0.004310
13     0.002947
14     0.002381
15     0.001651
19     0.001419
18     0.001323
17     0.001244
16     0.001158
21     0.000765
20     0.000665
23     0.000614
22     0.000500
26     0.000462
48     0.000445
24     0.000431
58     0.000397
27     0.000320
37     0.000265
33     0.000230
35     0.000184
29     0.000163
38     0.000153
25     0.000075
32     0.000069
28     0.000050
34     0.000048
42     0.000036
30     0.000023
52     0.000017
73     0.000017
60     0.000008
55     0.000004
50     0.000002
108    0.000002
Name: guest_count, dtype: float64

In [196]:
#Will only do for the hockey data since this is just for the code

In [197]:
data["order_time"] = pd.to_datetime(data["order_time"], format="%Y-%m-%d %H:%M:%S.%f")
data["order_item_time"] = pd.to_datetime(data["order_item_time"], format="%Y-%m-%d %H:%M:%S.%f")
data["order_time_closed"] = pd.to_datetime(data["order_time_closed"], format="%Y-%m-%d %H:%M:%S.%f")

/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [198]:
data = pd.concat([data, pd.get_dummies(data["category_id"], prefix="category_id")], axis=1)
# count total of orders per category id
category_ids = [float(i) for i in range(1, 7)]
for category_id in category_ids:
    data["total_orders_category_id_" + str(category_id)] =\
        data.groupby("order_id")["category_id_" + str(category_id)].transform("sum")

In [199]:
total_categories_ids = ["total_orders_category_id_" + str(float(i)) for i in range(1, 7)]
total_categories_ids_to_sum = [column for column in total_categories_ids if column != "total_orders_category_id_5.0"]
data["total_orders"] = data.apply(lambda order: sum(order[column] for column in total_categories_ids_to_sum), axis=1)


In [200]:
data["order_day_of_week"] = data.order_time.apply(lambda ticket: ticket.day_name())


# 4. Add `period_of_day` feature ('breakfast', 'lunch', 'afternoon', 'dinner', 'night')
data["order_hour"] = data.order_time.apply(lambda ticket: ticket.hour)

def period_of_day(hour):
    if hour >= 6 and hour < 11: return 'breakfast'
    elif hour >= 11 and hour < 14: return 'lunch'
    elif hour >= 14 and hour < 18: return 'afternoon'
    elif hour >= 18 and hour < 21: return 'dinner'
    elif hour >= 21 and hour < 23: return 'hang_out'
    elif hour >= 23 or hour < 6: return 'night'

data["period_of_day"] = data.apply(lambda order: period_of_day(int(order["order_hour"])), axis=1)
#data.drop("order_hour", axis=1, inplace=True)

# 5. Add `is_weekend` feature
weekend = ["Friday", "Saturday", "Sunday"]
data["is_weekend"] = data.order_day_of_week.apply(lambda ticket_day: ticket_day in weekend)

In [162]:
sauces = set()
sub = set()
# drop unrelevant titles
def filter_titles(row):
    title = row["title"].lower()
    price = row["sales_before_tax"]

    if "xtra" in title:
        return False
    if "sub" in title and price == 0:
        sub.add(title)
        return False
    elif "add" in title:
        return False
    elif "sauce" in title:
        for t in foods_w_sauce: 
            if t in title: return True
        for t in remove_w_sauce:
            if t in title: return False
        sauces.add(title)
        return False
    elif "no " in title:
        return False
    elif "no." in title:
        return False
    elif "-no " in title:
        return False
    elif "side " in title:
        return False
    elif "+" in title:
        return False
    elif "dip" in title:
        return False
    elif "blue cheese" in title:
        return False
    elif "bbq" in title:
        return False
    elif "n/c" in title:
        return False
    elif "s/o" in title:
        return False
    elif title == '' or title == 'garlic.aioli' or title == 'gluten' or title == 'hot n honey' or title == 'honey garlic' or title == 'kids.' or title == 'to go':
        return False
    else:
        return True

foods_w_sauce = ["fingers", "spaghetti", "poutine", "wings", "pate", "bowl", "fries", "rigatoni", "pasta",
                "linguini", "frite"]
remove_w_sauce = ["no wing", "for wing", "on", "side"]
whitelist = set('.abcdefghijklmnopqrstuvwxyz \\//')
    
 

In [201]:
data.title = data.title.apply(lambda x: str(x))
data = data[data.apply(lambda x: filter_titles(x), axis=1)]


In [165]:
   

sharable_foods = [
    "pizza", "cake", "hot pot", "nachos", "guac", "wings", "Focaccia", "bread", "fries", "pretzels",
"quesadilla", "nuts", "fondue", "calamari", "fingers sauced","chicken fingers", "chkn fingers", "quesa stack" 
]

def find_sharable(title):
    title = " ".join(title.lower().split("."))
    for s in sharable_foods:
        if s in title:
            return 1
    return 0

data["sharable"] = data.title.apply(lambda x: find_sharable(x))




# meal with kids
data["kids_meal"] = data.title.apply(lambda x: 1 if 'kid' in x else 0)
# birthday
data["birthday"] = data.title.apply(lambda x: 1 if 'birthday' in x else 0)


/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/danielastepanov/miniconda3/envs/ds/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [202]:
data["order_minute"] = data.order_time.apply(lambda ticket: ticket.minute)
data["order_item_minute"] = data.order_item_time.apply(lambda ticket: ticket.minute)
data["order_item_hour"] = data.order_item_time.apply(lambda ticket: ticket.hour)
data["order_close_minute"] = data.order_time_closed.apply(lambda ticket: ticket.minute)
data["order_close_hour"] = data.order_time_closed.apply(lambda ticket: ticket.hour)

In [86]:
data.order_time[1]

Timestamp('2019-04-16 12:01:00')

In [95]:
d = (data.order_time[14] - data.order_time[1])

In [101]:
d.components.minutes

49

In [97]:
d / pd.Timedelta('1 hour')

7.816666666666666

In [269]:
import math

def find_total_time(data):
    order_ids = list(data.order_id.value_counts().keys())
    total_dict = {}
    for order in order_ids:
        total_dict[order] = {}
        df_order = data[data.order_id == order]

        closing_time = list(df_order.order_time_closed)[0]
        item_times = list(df_order.order_item_time.sort_values())
        earliest_ordering_time = item_times[0]
        order_time = list(df_order.order_time)[0]
        
        if order_time < earliest_ordering_time: 
            total_time = closing_time - order_time
            earliest_time = order_time
        else: 
            total_time = closing_time - earliest_ordering_time
            earliest_time = earliest_ordering_time

        total_dict[order]["total_time"] = total_time / pd.Timedelta('1 hour')

        #Calculating mealflow
        prev = 0
        prev_step = 0
        prev_meal_flow_step = 0
        total_dict[order]["meal_step"] = {}
        total_dict[order]["meal_flow"] = {}
        total_steps = 0
        for i,item in enumerate(item_times):
            #calculating meal step (if there are four orders all at the same time, they will be given 0 - 3 steps. If
            # all the orders were ordered at the same time, they will be given the same step: 0)
            
            
            #Calculating meal flow step - each meal flow step is 4 minutes long.
            if prev == 0:
                total_dict[order]["meal_step"][item] = 1
                prev_step = 1
                diff = (item - earliest_time)
                flow_step = math.floor(((diff.components.hours*60) + diff.components.minutes) /4)
                prev_meal_flow_step = flow_step
                total_dict[order]["meal_flow"][item] = flow_step
                total_dict[order]["meal_flow_beginning"] = flow_step
                prev = item
                continue

            if item == prev:
                continue

            
            diff = item - prev
            if ((diff.components.hours*60) + diff.components.minutes) >= 4:
                total_dict[order]["meal_step"][item] = prev_step + 1
            else:
                total_dict[order]["meal_step"][item] = prev_step
            prev_step = total_dict[order]["meal_step"][item]
            flow_step = math.floor(((diff.components.hours*60) + diff.components.minutes) /4) + prev_meal_flow_step
            total_dict[order]["meal_flow"][item] = flow_step
            prev_meal_flow_step = flow_step 
            prev = item
        
        diff = closing_time - item_times[-1]
        last_flow_step = math.floor(((diff.components.hours*60) + diff.components.minutes) /4)
        total_steps = last_flow_step + prev_meal_flow_step
        total_dict[order]["meal_flow_last_to_close"] = last_flow_step
        total_dict[order]["total_meal_flow_steps"] = total_steps

     
    return total_dict

total_dict = find_total_time(data)

In [270]:
data["total_order_time"] = data.order_id.apply(lambda x: total_dict[x]["total_time"])

In [271]:
data["meal_step"] = data.apply(lambda x: total_dict[x.order_id]["meal_step"][x.order_item_time], axis=1)

In [272]:
data["meal_flow_last_to_close"] = data.apply(lambda x: total_dict[x.order_id]["meal_flow_last_to_close"], axis=1)

In [273]:
data["total_meal_flow_steps"] = data.order_id.apply(lambda x: total_dict[x]["total_meal_flow_steps"])

In [274]:
data["meal_flow_step"] = data.apply(lambda x: total_dict[x.order_id]["meal_flow"][x.order_item_time], axis=1)

In [181]:
#data.to_csv("hoc_features.csv", index=False)

In [184]:
data.columns

Index(['bar_id', 'order_id', 'order_time', 'order_time_closed',
       'order_item_time', 'order_item_id', 'item_qty', 'sales_before_tax',
       'sales_inc_tax', 'guest_count', 'title', 'category_id', 'beer_brand_id',
       'beer_serving_type_id', 'beer_volume', 'category_id_1.0',
       'category_id_2.0', 'category_id_3.0', 'category_id_4.0',
       'category_id_5.0', 'category_id_6.0', 'total_orders_category_id_1.0',
       'total_orders_category_id_2.0', 'total_orders_category_id_3.0',
       'total_orders_category_id_4.0', 'total_orders_category_id_5.0',
       'total_orders_category_id_6.0', 'total_orders', 'order_day_of_week',
       'order_hour', 'period_of_day', 'is_weekend', 'sharable', 'kids_meal',
       'birthday', 'order_minute', 'order_item_minute', 'order_item_hour',
       'order_close_minute', 'order_close_hour', 'total_order_time',
       'meal_step', 'meal_flow_step', 'last_step'],
      dtype='object')

In [275]:
data.head(200)[100:120]

,bar_id,order_id,order_time,order_time_closed,order_item_time,order_item_id,item_qty,sales_before_tax,sales_inc_tax,guest_count,...,order_item_hour,order_close_minute,order_close_hour,total_order_time,meal_step,meal_flow_step,last_step,total_meal_flow_steps,closing_meal_flow_step,meal_flow_last_to_close
138,1177,520217262,2019-04-16 21:10:00,2019-04-16 23:01:05,2019-04-16 21:15:15,52021726202,1.0,20.74,23.442935,1,...,21,1,23,1.851389,2,1,2019-04-16 21:15:15,27,13,26
140,1177,520217262,2019-04-16 21:10:00,2019-04-16 23:01:05,2019-04-16 21:15:15,52021726201,1.0,0.00,0.000000,1,...,21,1,23,1.851389,2,1,2019-04-16 21:15:15,27,13,26
141,1177,520091650,2019-04-16 15:12:00,2019-04-16 16:20:46,2019-04-16 15:12:53,52009165000,1.0,17.97,20.309367,1,...,15,20,16,1.146111,1,0,2019-04-16 16:20:46,16,2,0
142,1177,520091650,2019-04-16 15:12:00,2019-04-16 16:20:46,2019-04-16 16:20:46,52009165002,0.5,3.96,4.475520,1,...,16,20,16,1.146111,2,16,2019-04-16 16:20:46,16,2,0
144,1177,520142383,2019-04-16 18:33:00,2019-04-16 20:07:20,2019-04-16 18:33:19,52014238300,1.0,0.00,0.000000,1,...,18,7,20,1.572222,1,0,2019-04-16 18:49:29,23,9,19
145,1177,520142383,2019-04-16 18:33:00,2019-04-16 20:07:20,2019-04-16 18:49:29,52014238301,1.0,15.99,18.440000,1,...,18,7,20,1.572222,2,4,2019-04-16 18:49:29,23,9,19
152,1177,520233357,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:17:54,52023335708,1.0,0.00,0.000000,2,...,21,19,23,2.212778,2,2,2019-04-16 23:04:20,29,5,3
153,1177,520233357,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:07:19,52023335701,1.0,18.09,20.441790,2,...,21,19,23,2.212778,1,0,2019-04-16 23:04:20,29,5,3
155,1177,520233357,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:07:19,52023335702,1.0,18.09,20.441790,2,...,21,19,23,2.212778,1,0,2019-04-16 23:04:20,29,5,3
158,1177,520233357,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 23:00:52,52023335711,1.0,9.21,10.407346,2,...,23,19,23,2.212778,4,26,2019-04-16 23:04:20,29,5,3


In [278]:
data[data.order_id == 520233357][['order_time', 'order_time_closed',
       'order_item_time','item_qty','guest_count', 'title', 
      'total_order_time','sales_before_tax','category_id',
       'meal_step', 'meal_flow_step', 'last_step', 'total_meal_flow_steps', "meal_flow_last_to_close"]]

,order_time,order_time_closed,order_item_time,item_qty,guest_count,title,total_order_time,sales_before_tax,category_id,meal_step,meal_flow_step,last_step,total_meal_flow_steps,meal_flow_last_to_close
152,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:17:54,1.0,2,Mashed,2.212778,0.00,2.0,2,2,2019-04-16 23:04:20,29,3
153,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:07:19,1.0,2,R.STERL.VN.CS.1z,2.212778,18.09,6.0,1,0,2019-04-16 23:04:20,29,3
155,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:07:19,1.0,2,R.STERL.VN.CS.1z,2.212778,18.09,6.0,1,0,2019-04-16 23:04:20,29,3
158,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 23:00:52,1.0,2,EXTREME.CAKE,2.212778,9.21,2.0,4,26,2019-04-16 23:04:20,29,3
160,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 23:04:20,1.0,2,HOT.TEA*,2.212778,3.99,4.0,4,26,2019-04-16 23:04:20,29,3
161,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 22:37:06,1.0,2,COFFEE*,2.212778,3.99,4.0,3,21,2019-04-16 23:04:20,29,3
163,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:17:54,1.0,2,Mashed,2.212778,0.00,2.0,2,2,2019-04-16 23:04:20,29,3
164,2019-04-16 21:07:00,2019-04-16 23:19:46,2019-04-16 21:07:19,1.0,2,R.STERL.VN.CS.1z,2.212778,18.09,6.0,1,0,2019-04-16 23:04:20,29,3


In [229]:
data.order_time_closed[640160]

Timestamp('2019-04-20 14:20:05')

In [238]:
data.order_item_time[640160]

Timestamp('2019-04-20 12:41:12')

In [257]:
(data.order_item_time[640160] - data.order_item_time[640158]) /pd.Timedelta('1 hour')

0.26666666666666666